<a href="https://colab.research.google.com/github/avc9/NLPlay-with-Transformers--SoC-2021/blob/main/BertModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
data=pd.read_csv("/content/drive/MyDrive/SOC-2021 data/IMDB Dataset.csv")

In [5]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_hypen(text):
  return re.sub('-','',text)


def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text=remove_hypen(text)
    return text

data['review_processed']=data['review'].apply(denoise_text)
print(data.head(5))

                                              review  ...                                   review_processed
0  One of the other reviewers has mentioned that ...  ...  One of the other reviewers has mentioned that ...
1  A wonderful little production. <br /><br />The...  ...  A wonderful little production. The filming tec...
2  I thought this was a wonderful way to spend ti...  ...  I thought this was a wonderful way to spend ti...
3  Basically there's a family where a little boy ...  ...  Basically there's a family where a little boy ...
4  Petter Mattei's "Love in the Time of Money" is...  ...  Petter Mattei's "Love in the Time of Money" is...

[5 rows x 3 columns]


In [6]:
data['sentiment']=data['sentiment'].map({'positive':1,'negative':0})
print(data["sentiment"][:5])

0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: int64


In [7]:
from datasets import Dataset

data=Dataset.from_pandas(data)

In [8]:
print(data)

Dataset({
    features: ['review', 'sentiment', 'review_processed'],
    num_rows: 50000
})


In [9]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples["review_processed"], padding="max_length", truncation=True,return_token_type_ids=False,max_length=200)

tokenized_datasets = data.map(tokenize_function, batched=True)

In [10]:
print(tokenized_datasets)

Dataset({
    features: ['attention_mask', 'input_ids', 'review', 'review_processed', 'sentiment'],
    num_rows: 50000
})


In [11]:
tokenized_datasets = tokenized_datasets.remove_columns(["review","review_processed"])
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")

In [12]:
print(tokenized_datasets)

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 50000
})


In [13]:
tokenized_datasets=tokenized_datasets.train_test_split(test_size=0.2)
train_data=tokenized_datasets['train']
test_data=tokenized_datasets['test']

In [14]:
train_data.set_format("torch")
test_data.set_format("torch")

In [15]:
print(train_data)

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 40000
})


In [16]:
"""class CustomDataset(Dataset):

  def __init__ (self,df):
    self.attention=df['attention_mask']
    self.ip_ids=df['input_ids']
    self.labels=df["labels"]
    

  def __len__(self):
    return ip_ids.shape[0]

  def __getitem__(self,x):
    
    return ip_ids,attention,labels

ds=CustomDataset(tokenized_datasets)
full_dataloader=DataLoader(ds,batch_size=32)"""

'class CustomDataset(Dataset):\n\n  def __init__ (self,df):\n    self.attention=df[\'attention_mask\']\n    self.ip_ids=df[\'input_ids\']\n    self.labels=df["labels"]\n    \n\n  def __len__(self):\n    return ip_ids.shape[0]\n\n  def __getitem__(self,x):\n    \n    return ip_ids,attention,labels\n\nds=CustomDataset(tokenized_datasets)\nfull_dataloader=DataLoader(ds,batch_size=32)'

In [17]:
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=32)
eval_dataloader = DataLoader(test_data, batch_size=32)
#full_dataloader=DataLoader(tokenized_datasets,batch_size=32)

In [18]:
#full_dataloader=DataLoader(tokenized_datasets,batch_size=32)

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [20]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [21]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [22]:
loss_fn=nn.CrossEntropyLoss()

In [23]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model=model.to(device)
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768)

In [24]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    i=0
    for batch in train_dataloader:
        i+=1
        ips=batch['input_ids'].to(device)
        at=batch['attention_mask'].to(device)
        lbls=batch['labels'].to(device)
        #batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(
          input_ids=ips,
          attention_mask=at
        )
        loss = loss_fn(outputs.logits,lbls)
        if i%100==0:
          print(f'Batchs {i} : loss={loss.item()}')
        loss.backward()

        optimizer.step()
        #lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Batchs 100 : loss=0.7280068397521973
Batchs 200 : loss=0.7195586562156677
Batchs 300 : loss=0.6792771816253662
Batchs 400 : loss=0.7403715252876282
Batchs 500 : loss=0.4994155764579773
Batchs 600 : loss=0.6747782230377197
Batchs 700 : loss=0.49341893196105957
Batchs 800 : loss=0.45201191306114197
Batchs 900 : loss=0.4349404275417328
Batchs 1000 : loss=0.41780656576156616
Batchs 1100 : loss=0.5307081937789917
Batchs 1200 : loss=0.3673816919326782
Batchs 100 : loss=0.5176873803138733
Batchs 200 : loss=0.5415432453155518
Batchs 300 : loss=0.43107330799102783
Batchs 400 : loss=0.47166839241981506
Batchs 500 : loss=0.40786266326904297
Batchs 600 : loss=0.24705491960048676
Batchs 700 : loss=0.3744812607765198
Batchs 800 : loss=0.2559586465358734
Batchs 900 : loss=0.3125087022781372
Batchs 1000 : loss=0.35304099321365356
Batchs 1100 : loss=0.3127206861972809
Batchs 1200 : loss=0.2636920213699341
Batchs 100 : loss=0.2872583568096161
Batchs 200 : loss=0.19434508681297302
Batchs 300 : loss=0.304

In [27]:
from datasets import load_metric
metric= load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8577}

In [26]:
torch.save(model,'/content/drive/MyDrive/model1.pt')